In [ ]:
%reload_ext postcell
%postcell register

# Decorators

Python is among several languages which provides a way to _annotate_ or _decorate_ functions. This is an extremely powerful feature, although not always without caveats. Before we get into decorators, we need to review some concepts from the 'higher order functions' lecture.

Recall that functions are normal objects

In [ ]:
def myfunc(): return 10

In [ ]:
myfunc.__name__

In [ ]:
myfunc()

In [ ]:
x = myfunc
x()

They can be passed into function

In [ ]:
def double_me(n): return n * 2

double_me(10)

In [ ]:
list(map(double_me, [1,2,3,4,5,6,7,8,9]))

Functions can even be returned from functions

In [ ]:
def n_adder(n):
    def adder(value):
        return n + value
    
    return adder

In [ ]:
five_adder = n_adder(5)

In [ ]:
five_adder(1)

In [ ]:
five_adder(2)

`five_adder` isn't the most useful function however. Soon we will see some useful examples, but let's also review `*args` and `*kwargs`

#### `*args` and `**kwargs` review

We saw examples of functions which either take no arguments or a single argument. We _know_ how many arguments it takes. What if we didn't know how many arguments a function takes? We can use the `*args` to pass in arguments as a list and have Python convert them to proper function parameters

`pow` takes two arguments

In [ ]:
pow(10,2)

Obviously, it does NOT take a list with two arguments

In [ ]:
pow([10, 2]) 

However, by using the `*` before a list, we can tell Python to conver the elements in the list to function parameters

In [ ]:
pow(*[10,2]) 

Similarly, we can pass named parameters to the `open` function

In [ ]:
open(file='../../postcell.conf.bak', mode='r').readlines()

Obviously we can't pass in a dictionary of named parameters

In [ ]:
open({'file':'../../postcell.conf.bak', 'mode':'r'}).readlines()

But, we can tell Python to convert this dictionary to named parameters by prefixing the dictionary with `**`

In [ ]:
open(**{'file':'../../postcell.conf.bak', 'mode':'r'}).readlines()

Given what we have reviewd, we can now write a couple of very useful function wrappers

#### timer wrapper

In [ ]:
import time

def timer(f, *args, **kwargs):
    start = time.time()
    rslt = f(*args, **kwargs)
    end = time.time()
    print(f"Function {f.__name__} took {end-start} seconds to execute")
    return rslt

In [ ]:
timer(open, **{'file':'../../postcell.conf.bak', 'mode':'r'})

In [ ]:
timer(lambda x: sum(range(x)), 10_000_000)

#### logger wrapper

In [ ]:
def logger(f, *args, **kwargs):
    print(f"Starting execution of function {f.__name__}")
    rslt = f(*args, **kwargs)
    print(f"Finished execution of function {f.__name__}")
    return rslt

In [ ]:
logger(open, **{'file':'../../postcell.conf.bak', 'mode':'r'})

#### A better integrated logger
The logger above is very useful, but it changes the syntax of Python. Every function needs to be wrapped in the `logger` function. What if we could continue to call functions as before, but also get the benefits of wrapper functions?

In [ ]:
def logger2(f):
    def inner_func(*args, **kwargs):
        print(f"Starting execution of function {f.__name__}")
        rslt = f(*args, **kwargs)
        print(f"Finished execution of function {f.__name__}")
        return rslt
    return inner_func

In [ ]:
def say_hello(name): return f"Hello {name}"

In [ ]:
say_hello("Shahbaz")

In [ ]:
logger(say_hello, "Shahbaz")

In [ ]:
say_hello2 = logger2(say_hello)

In [ ]:
say_hello2("Shahbaz")

### Decorators
Decorators are a way in which certain annotated function are wrapped in other functions, before being called.

In [ ]:
@logger2
def say_bye(name):
    return f"Good bye {name}"

In [ ]:
say_bye("Shahbaz")

Wrappers are called, even if the wrapped function is not

In [ ]:
def wrapper(f):
    print(f"Just executed 'wrapper' for function {f.__name__}")


@wrapper
def hello():
    print("Just executed 'wrapper'")
    return "Hello"

#hello() #<= Notice that we have not run this yet!

Decorators with arguments behave unexpectedly

In [ ]:
def wrapper(p1, p2):
    print(f"Just executed 'wrapper' for arguments {p1} and {p2}")
    def inner_wrapper(f):
        print(f"Just executed 'inner_wrapper' for function {f.__name__}")
    return inner_wrapper
    

@wrapper("Homer", "Marge")
def hello():
    print("Just executed 'wrapper'")
    return "Hello"

#hello() #<= Notice that we have not run this yet!

You can wrap a function in multiple decorators

In [ ]:
def wrapper1(f):
    print(f"Just executed 'wrapper1' for function {f.__name__}")

def wrapper2(f):
    print(f"Just executed 'wrapper2' for function {f.__name__}")
    return f # <= Why do I have to return this function??


@wrapper1
@wrapper2 #<= This executes first and the output of this is passed to wrapper 2
def hello():
    print("Just executed 'wrapper'")
    return "Hello"

#hello() #<= Notice that we have not run this yet!

#### Decorator examples

##### Logger

In [ ]:
def log(f):
    def decorate(*args, **kwargs):
        print(f"Starting execution of function {f.__name__}")
        rslt = f(*args, **kwargs)
        print(f"Finished execution of function {f.__name__}")
        return rslt
    
    return decorate

In [ ]:
@log
def say_hello(name):
    return f"Hello {name}"

In [ ]:
say_hello("Shahbaz")

##### Timer

In [ ]:
import time

def timeit(f):
    def decorate(*args, **kwargs):
        start = time.time()
        rslt = f(*args, **kwargs)
        end = time.time()
        print(f"Function {f.__name__} took {end-start} seconds to execute")
        return rslt

    return decorate

In [ ]:
@timeit
def say_bye(name):
    return f"Bye {name}"

In [ ]:
say_bye("Shahbaz")

#### Cacher

In [ ]:
def cache(f):
    c = dict()
    def decorate(*args):
        # Warning: this will not work for functions which have keyword arguments
        v = c.get(args, None)
        
        if v:
            print("Cached value found, no need to call the function")
            return v
        else:
            rslt = f(*args)
            c[args] = rslt
        return rslt
    
    return decorate

In [ ]:
@cache
def say_howdy(name):
    print("Running howdy")
    return f"Howdy {name}"

In [ ]:
say_howdy("Shahbaz")

In [ ]:
say_howdy("Shahbaz")

In [ ]:
say_howdy("Shahbaz")

In [ ]:
say_howdy("Homer")

In [ ]:
say_howdy("Homer")

In [ ]:
say_howdy("Shahbaz")

**Exercise**  Write a decorator which executes functions twice. You should be able to call it as such:]

```python
@double
say_howdy('Marge')
```

In [ ]:
%%postcell exercise_025_245_a

#type your answer here

**Decorators are not always great**
The author of these notes is not always a fan of decorators. They can intorduce too much "magic." They break the normal flow of the way programs are read by programmers. Given a function, the developer can see the class it belongs to, follow the line by line logic and confirm the inputs and the outputs. A wrapper function can remove the ability to read code in a linear fashion. Developers should be careful when using decorators.

p.s.

Weirdly, some examples in this lecture are very similar to the examples in https://realpython.com/primer-on-python-decorators, although the author of these notes didn't reference that web page until after the initial draft.